In [1]:
import psycopg2 as pg
import pandas as pd
from prophet import Prophet

In [2]:
def connectionDataBase():
    connection = pg.connect(user="postgres", password="2023@Tag", host="159.65.42.225", port=5432, database="comercial_BI_Relational")
    print(connection)
    return connection

conn = connectionDataBase()
conn

<connection object at 0x0000017662B508C0; dsn: 'user=postgres password=xxx dbname=comercial_BI_Relational host=159.65.42.225 port=5432', closed: 0>


<connection object at 0x0000017662B508C0; dsn: 'user=postgres password=xxx dbname=comercial_BI_Relational host=159.65.42.225 port=5432', closed: 0>

In [6]:
conn = connectionDataBase()

cursor = conn.cursor()

query = '''SELECT cnae_principal_,data_incio_atividade from cnp_cnpj_ WHERE cnae_principal_ = 'Atividades de contabilidade' AND data_incio_atividade >= CURRENT_DATE - INTERVAL '5 years' '''
query=cursor.execute(query)
query=cursor.fetchall()

nomes_colunas = [desc[0] for desc in cursor.description]
df_size = pd.DataFrame(query,columns=nomes_colunas)
df_size

,cnae_principal_,data_incio_atividade
0,Atividades de contabilidade,2019-03-29
1,Atividades de contabilidade,2019-04-02
2,Atividades de contabilidade,2019-04-02
3,Atividades de contabilidade,2019-04-02
4,Atividades de contabilidade,2019-04-05
...,...,...
21729,Atividades de contabilidade,2023-12-27
21730,Atividades de contabilidade,2023-12-27
21731,Atividades de contabilidade,2023-12-27
21732,Atividades de contabilidade,2023-12-28


In [13]:
df_size['data_incio_atividade']=pd.to_datetime(df_size['data_incio_atividade'])

In [16]:
df2=df_size.sort_values('data_incio_atividade',ascending=False)
df2['Dif_Meses'] =df2['data_incio_atividade'].dt.to_period('M')
df_qtd = pd.DataFrame(df2[['cnae_principal_','Dif_Meses']].groupby('Dif_Meses').count()).reset_index()
df_qtd['Dif_Meses'] = pd.to_datetime(df_qtd['Dif_Meses'].astype(str) + '-01')
df_qtd['Dif_Meses']=pd.to_datetime(df_qtd['Dif_Meses'] + pd.offsets.MonthEnd(0),format='%Y-%m-%d')
df_qtd['Dif_Meses']=df_qtd['Dif_Meses'].astype(str)
df_qtd.columns=['Dif_Meses','count']
df_qtd['Dif_Meses']=pd.to_datetime(df_qtd['Dif_Meses'])
df_qtd=df_qtd.sort_values(by='Dif_Meses',ascending=True)

In [26]:
def create_df(cnae):
    conn = connectionDataBase()

    cursor = conn.cursor()
    query = f'''SELECT cnae_principal_,data_incio_atividade from cnp_cnpj_ WHERE cnae_principal_ = '{cnae}' AND data_incio_atividade >= CURRENT_DATE - INTERVAL '5 years' '''
    query=cursor.execute(query)
    query=cursor.fetchall()

    nomes_colunas = [desc[0] for desc in cursor.description]
    df_size = pd.DataFrame(query,columns=nomes_colunas)
    df2=df_size.sort_values('data_incio_atividade',ascending=False)
    df2['data_incio_atividade']=pd.to_datetime(df2['data_incio_atividade'])
    df2['Dif_Meses'] =df2['data_incio_atividade'].dt.to_period('M')
    df_qtd = pd.DataFrame(df2[['cnae_principal_','Dif_Meses']].groupby('Dif_Meses').count()).reset_index()
    df_qtd['Dif_Meses'] = pd.to_datetime(df_qtd['Dif_Meses'].astype(str) + '-01')
    df_qtd['Dif_Meses']=pd.to_datetime(df_qtd['Dif_Meses'] + pd.offsets.MonthEnd(0),format='%Y-%m-%d')
    df_qtd['Dif_Meses']=df_qtd['Dif_Meses'].astype(str)
    df_qtd.columns=['Dif_Meses','count']
    df_qtd['Dif_Meses']=pd.to_datetime(df_qtd['Dif_Meses'])
    df_qtd=df_qtd.sort_values(by='Dif_Meses',ascending=True)
    df_qtd.columns=['ds','y']
    return df_qtd

In [ ]:
for cnae in df_size['cna_name'].unique().tolist():
                df2=df_size[df_size['cna_name']==cnae].sort_values('data_incio_atividade',ascending=False)
                df2['Dif_Meses'] =df2['data_incio_atividade'].dt.to_period('M')

                df_qtd = pd.DataFrame(df2[['cna_name','Dif_Meses']].groupby('Dif_Meses').count()).reset_index()
                p_valor = df_qtd['cna_name'].iloc[0]
                r_valor = df_qtd['cna_name'].iloc[1:].sum()

                razao_s = r_valor - p_valor
                list_razao.append(razao_s)
                df_qtd['Dif_Meses'] = pd.to_datetime(df_qtd['Dif_Meses'].astype(str) + '-01')
                df_qtd['Dif_Meses']=pd.to_datetime(df_qtd['Dif_Meses'] + pd.offsets.MonthEnd(0),format='%Y-%m-%d')
                df_qtd['Dif_Meses']=df_qtd['Dif_Meses'].astype(str)
                df_qtd.columns=['Dif_Meses','count']

                print(df_qtd)

In [31]:
def make_forecast(cnae):
    m = Prophet(seasonality_mode='multiplicative').fit(create_df(cnae))
    future = m.make_future_dataframe(periods=6, freq = "M")
    forecast = m.predict(future)
   
    df = forecast[['ds','yhat','yhat_lower', 'yhat_upper']]
    date_range = df['ds']
    df['ds']=df['ds'].dt.to_period('M')
    return df

In [32]:
df=make_forecast('Atividades de contabilidade')

12:25:55 - cmdstanpy - INFO - Chain [1] start processing
12:25:55 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\saulo\AppData\Local\Programs\Python\Python311\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\saulo\AppData\Local\Temp\ipykernel_47840\704386809.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ds']=df['ds'].dt.to_period('M')


In [33]:
df

,ds,yhat,yhat_lower,yhat_upper
0,2019-02,309.992476,235.760858,392.510098
1,2019-03,402.947747,326.118799,478.671909
2,2019-04,353.396347,276.869766,426.853346
3,2019-05,364.841400,290.908357,440.680615
4,2019-06,346.788182,271.608471,425.942148
...,...,...,...,...
61,2024-03,363.931587,285.799976,447.631450
62,2024-04,263.812646,190.591408,345.877622
63,2024-05,344.873479,270.887456,420.080903
64,2024-06,373.544934,299.268711,456.885908


In [34]:
df['Dif_Meses'] = pd.to_datetime(df['ds'].astype(str) + '-01')

In [35]:
df

,ds,yhat,yhat_lower,yhat_upper,Dif_Meses
0,2019-02,309.992476,235.760858,392.510098,2019-02-01
1,2019-03,402.947747,326.118799,478.671909,2019-03-01
2,2019-04,353.396347,276.869766,426.853346,2019-04-01
3,2019-05,364.841400,290.908357,440.680615,2019-05-01
4,2019-06,346.788182,271.608471,425.942148,2019-06-01
...,...,...,...,...,...
61,2024-03,363.931587,285.799976,447.631450,2024-03-01
62,2024-04,263.812646,190.591408,345.877622,2024-04-01
63,2024-05,344.873479,270.887456,420.080903,2024-05-01
64,2024-06,373.544934,299.268711,456.885908,2024-06-01


In [37]:
import plotly.graph_objects as go

# Dados de exemplo
x = [1, 2, 3, 4, 5]
y = [10, 15, 13, 18, 16]

# Criar a figura
fig = go.Figure()

# Adicionar o gráfico de dispersão à figura
fig.add_trace(go.Scatter(x=df['Dif_Meses'], y=df['yhat'], mode='markers', name='Pontos'))

# Personalizar o layout (opcional)
fig.update_layout(title='Previsão IA abertura de novas empresas',
                  xaxis_title='Data',
                  yaxis_title='Quantidade',
            )

# Mostrar o gráfico
fig.show()


In [20]:
def concat(mat):
    df_1,_,_,date_range = make_forecast(mat,12)
    df_2 = create_df(mat).reset_index().drop('index',axis = 1)
    df = pd.concat([df_2,df_1],axis=1)
    df['ds']=date_range
    return df